## This notebook aims to find out how accurate EfficientNet-b0 with Face recognition weights can converge when entirely trained on OZON-AffectNet composite dataset. Expressnion heah is mounted on 512 logits

In [1]:
import re
import torchvision
from torchvision import datasets, models, transforms
from torchvision.datasets.folder import default_loader
import math
import collections
from functools import partial
import random
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch import nn
from torch.nn import functional as F
import copy
from torch.utils import model_zoo
from torch.nn import Sequential, BatchNorm1d, BatchNorm2d, Dropout, Module, Linear
import yaml
import argparse
import os
import time
import sys
device = torch.device("cuda:5" if torch.cuda.is_available() else "cpu")

## Model initialzation

In [2]:
conf = argparse.ArgumentParser(description='traditional_training for face recognition.')

conf.add_argument("--backbone_type", type = str,default = 'EfficientNet',
                      help = "Mobilefacenets, Resnet.")
conf.add_argument("--backbone_conf_file", type = str ,default ='/storage_labs/3030/BelyakovM/Face_attributes/Code/EfficientNet_B0_face_recognizer/FaceX-Zoo/training_mode/backbone_conf.yaml', 
                      help = "the path of backbone_conf.yaml.")
conf.add_argument("--head_type", type = str ,default = 'AdaM-Softmax', 
                      help = "mv-softmax, arcface, npc-face.")
conf.add_argument("--head_conf_file", type = str ,default = '/storage_labs/3030/BelyakovM/Face_attributes/Code/EfficientNet_B0_face_recognizer/FaceX-Zoo/training_mode/head_conf.yaml', 
                      help = "the path of head_conf.yaml.")
    
args = conf.parse_args([])

In [3]:
def load_pretrained_weights(model, model_name, weights_path=None, load_fc=True, advprop=False):
    """Loads pretrained weights from weights path or download using url.
    Args:
        model (Module): The whole model of efficientnet.
        model_name (str): Model name of efficientnet.
        weights_path (None or str):
            str: path to pretrained weights file on the local disk.
            None: use pretrained weights downloaded from the Internet.
        load_fc (bool): Whether to load pretrained weights for fc layer at the end of the model.
        advprop (bool): Whether to load pretrained weights
                        trained with advprop (valid when weights_path is None).
    """
    if isinstance(weights_path, str):
        state_dict = torch.load(weights_path)['state_dict']
        for key_name in list(state_dict.keys()):
            new_key = key_name.replace('backbone.','')
            state_dict[new_key] = state_dict.pop(key_name)
    else:
        # AutoAugment or Advprop (different preprocessing)
        url_map_ = url_map_advprop if advprop else url_map
        state_dict = model_zoo.load_url(url_map_[model_name])

    if load_fc:
        state_dict.pop('head.weight')
        ret = model.load_state_dict(state_dict, strict=False)
        assert not ret.missing_keys, 'Missing keys when loading pretrained weights: {}'.format(ret.missing_keys)
    else:
        state_dict.pop('backbone._fc.weight')
        state_dict.pop('backbone._fc.bias')
        ret = model.load_state_dict(state_dict, strict=False)
        assert set(ret.missing_keys) == set(
            ['_fc.weight', '_fc.bias']), 'Missing keys when loading pretrained weights: {}'.format(ret.missing_keys)
    assert not ret.unexpected_keys, 'Missing keys when loading pretrained weights: {}'.format(ret.unexpected_keys)

    print('Loaded pretrained weights for {}'.format(model_name))

In [4]:
sys.path.append('/storage_labs/3030/BelyakovM/Face_attributes/Code/EfficientNet_B0_face_recognizer/FaceX-Zoo')
from backbone.backbone_def import BackboneFactory
from head.head_def import HeadFactory
class FaceModel(torch.nn.Module):
    """Define a traditional face model which contains a backbone and a head.
    
    Attributes:
        backbone(object): the backbone of face model.
        head(object): the head of face model.
    """
    def __init__(self, backbone_factory, head_factory):
        """Init face model by backbone factorcy and head factory.
        
        Args:
            backbone_factory(object): produce a backbone according to config files.
            head_factory(object): produce a head according to config files.
        """
        super(FaceModel, self).__init__()
        self.backbone = backbone_factory.get_backbone()
        self.head = head_factory.get_head()
        self.expression_head = Sequential(nn.Linear(in_features=512, out_features=30, bias=True),nn.Linear(in_features=30, out_features=7, bias=True))

    def forward(self, data):
        logits = self.backbone.forward(data)
        expression = self.expression_head(logits)
       
        return logits,expression
backbone_factory = BackboneFactory(args.backbone_type, args.backbone_conf_file)   
head_factory = HeadFactory(args.head_type, args.head_conf_file)
efficientnet_b0_pretrained_entirely_expressionhead2logits = FaceModel(backbone_factory, head_factory)
load_pretrained_weights(efficientnet_b0_pretrained_entirely_expressionhead2logits.backbone,args.backbone_type,weights_path ='/storage_labs/3030/BelyakovM/Face_attributes/Code/EfficientNet_B0_face_recognizer/efficientnet_facerecognition_weights.pt',load_fc=True )
efficientnet_b0_pretrained_entirely_expressionhead2logits=efficientnet_b0_pretrained_entirely_expressionhead2logits.to(device)

/storage_labs/3030/BelyakovM/Face_attributes/Code/EfficientNet_B0_face_recognizer/FaceX-Zoo/backbone/backbone_def.py:32: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  backbone_conf = yaml.load(f)
/storage_labs/3030/BelyakovM/Face_attributes/Code/EfficientNet_B0_face_recognizer/FaceX-Zoo/head/head_def.py:32: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  head_conf = yaml.load(f)


backbone param:
{'width': 1.0, 'depth': 1.0, 'image_size': 112, 'drop_ratio': 0.2, 'out_h': 7, 'out_w': 7, 'feat_dim': 512}
head param:
{'feat_dim': 512, 'num_class': 72778, 'scale': 32, 'lamda': 70.0}
Loaded pretrained weights for EfficientNet


In [5]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(112),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(120),
        transforms.CenterCrop(112),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [6]:
from Datasets.MyDataset_expression_AffectPartly import MyDataset_expression_AffectPartly
    
annotations_path = '/storage_labs/3030/BelyakovM/Face_attributes/ds/db_BuevichP/emochon/AffecetNet_partly/original/annotation_'
image_datasets_AffectPartly = {x: MyDataset_expression_AffectPartly(annotations_path,x,{'neutral':0, 'happiness':1, 'sadness':2, 'surprise':3, 'anger':4, 'disgust':5,'fear':6},
                                          data_transforms[x])
                  for x in ['train', 'val']}

In [7]:
from Datasets.MyDataset_expression_OZON import MyDataset_expression_OZON
    
annotations_path_ozon = '/storage_labs/3030/BelyakovM/Face_attributes/ds/db_BuevichP/emochon/OZON_expressions_dataset/train_7expressions_annotation.txt'
image_datasets_ozon = {x: MyDataset_expression_OZON(annotations_path_ozon,x,{'neutral':0, 'happy':1, 'sad':2, 'surprise':3, 'anger':4, 'disgust':5,'fear':6},
                                          data_transforms[x])
                  for x in ['train', 'val']}

In [8]:
image_datasets_AffectPartly_OZON = {'train': torch.utils.data.ConcatDataset([image_datasets_AffectPartly['train'],image_datasets_ozon['train']]),
                                    'val_AffectPartly': image_datasets_AffectPartly['val'],'val_ozon':image_datasets_ozon['val']}
dataloaders_AffectPartly_OZON = {x: torch.utils.data.DataLoader(image_datasets_AffectPartly_OZON[x], batch_size=63,
                                             shuffle=True, num_workers=4) for x in ['train', 'val_AffectPartly','val_ozon']}

dataset_sizes_AffectPartly_OZON = {x: image_datasets_AffectPartly_OZON[x].__len__() for x in ['train', 'val_AffectPartly','val_ozon']}


In [9]:
def train_model(model, criterion, optimizer, scheduler,dataloaders,dataset_sizes, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val_AffectPartly','val_ozon']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval() 
            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)[1]
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val_ozon' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [10]:
criterion = nn.CrossEntropyLoss()

optimizer_efficientnet_b0_pretrained_entirely_expressionhead2logits = optim.SGD(efficientnet_b0_pretrained_entirely_expressionhead2logits.parameters(), lr=0.00001, momentum=0.9)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_efficientnet_b0_pretrained_entirely_expressionhead2logits, step_size=7, gamma=0.1)

In [11]:

efficientnet_b0_pretrained_entirely_expressionhead2logits = train_model(efficientnet_b0_pretrained_entirely_expressionhead2logits, criterion,optimizer_efficientnet_b0_pretrained_entirely_expressionhead2logits, exp_lr_scheduler,dataloaders_AffectPartly_OZON,dataset_sizes_AffectPartly_OZON,
                       num_epochs=30)

Epoch 0/29
----------
train Loss: 1.9678 Acc: 0.1207
val_AffectPartly Loss: 1.9554 Acc: 0.1537
val_ozon Loss: 1.9694 Acc: 0.1254

Epoch 1/29
----------
train Loss: 1.9657 Acc: 0.1244
val_AffectPartly Loss: 1.9533 Acc: 0.1529
val_ozon Loss: 1.9662 Acc: 0.1291

Epoch 2/29
----------
train Loss: 1.9645 Acc: 0.1224
val_AffectPartly Loss: 1.9514 Acc: 0.1529
val_ozon Loss: 1.9631 Acc: 0.1348

Epoch 3/29
----------
train Loss: 1.9623 Acc: 0.1299
val_AffectPartly Loss: 1.9493 Acc: 0.1589
val_ozon Loss: 1.9598 Acc: 0.1398

Epoch 4/29
----------
train Loss: 1.9602 Acc: 0.1304
val_AffectPartly Loss: 1.9472 Acc: 0.1591
val_ozon Loss: 1.9565 Acc: 0.1442

Epoch 5/29
----------
train Loss: 1.9594 Acc: 0.1337
val_AffectPartly Loss: 1.9457 Acc: 0.1617
val_ozon Loss: 1.9541 Acc: 0.1500

Epoch 6/29
----------
train Loss: 1.9566 Acc: 0.1377
val_AffectPartly Loss: 1.9440 Acc: 0.1626
val_ozon Loss: 1.9516 Acc: 0.1523

Epoch 7/29
----------
train Loss: 1.9559 Acc: 0.1404
val_AffectPartly Loss: 1.9439 Acc: 0.

In [12]:
torch.save(efficientnet_b0_pretrained_entirely_expressionhead2logits.state_dict(), '/storage_labs/3030/BelyakovM/Face_attributes/Saved_models/efficientnet_b0_pretrained_entirely_trained_expressionhead2logits.pth')